El jugador más “retador” y el jugador más “retado

La versión en sql: 
```sql
SELECT
    ID_Jugador_Retador,
    COUNT(ID_Jugador_Retado) AS Numero_Retos_Recibidos
FROM
    jugadores
GROUP BY
    ID_Jugador_Retado
ORDER BY
    Numero_Retos_Recibidos DESC
LIMIT 1;
```
Lo traducimos como dos jobs, el primero que cuenta las ocurrencias del id_Jugador_Retador y el segundo que sacá el maximo de estos 

In [ ]:
from MRE import Job
inputDir = "./jugadores/"
outputDir = "./retadoresContados/"

In [ ]:
# Un solo Job que cuenta tanto retadores como retados
# La clave de salida es una tupla (TYPE, player_id) donde TYPE es 'RETADOR' o 'RETADO'
# El valor de salida es la cantidad de veces que ese player_id aparece como retador o retado
def fmap(key, value, context):
    # por cada linea genero dos con clave tipo + id y valor 1
    retador = int(key)
    parts = value.split('	')
    context.write(('RETADOR', retador), 1)
    retado = int(parts[0])
    context.write(('RETADO', retado), 1)

def fcombiner(key, values, context):
    # simple combiner: sum counts locally
    s = 0
    for v in values:
        s += int(v)
    context.write(key, s)

def fred(key, values, context):
    # reducer to sum counts per (TYPE, player_id)
    s = 0
    for v in values:
        s += int(v)
    # write the composite key (TYPE, player) and the total count
    context.write(key, s)

jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCountBoth.setCombiner(fcombiner)
success = jobCountBoth.waitForCompletion()

print(success)

ahora saco el maximo

In [ ]:
# The second job reads the (TYPE,player) counts produced by the first job
# Each line in the first job's output looks like: TYPE	player	count
inputDir = "./retadoresContados/"
outputDir = "./output2/"

In [ ]:
# Mapper for the max-job: pass through so reducer receives TYPE as key
def fmap_max(key, value, context):
    # When reading the output of the counting job, the framework gives
    # key = TYPE (e.g. 'RETADOR' or 'RETADO') and value = 'player	count'
    context.write(key, value)

def fred_max(key, values, context):
    # values is an iterator of strings like 'player	count'
    max_count = -1
    max_player = None
    for v in values:
        parts = v.split('	')
        player = int(parts[0])
        cnt = int(parts[1])
        if cnt > max_count:
            max_count = cnt
            max_player = player
    # write TYPE -> (player_with_max, max_count)
    context.write(key, (max_player, max_count))

jobMax = Job(inputDir, outputDir, fmap_max, fred_max)
# no combiner needed for the max job
success = jobMax.waitForCompletion()

print(success)